In [ ]:
from IPython.core.display import display, HTML
import yaml

In [ ]:
thingToUpdate = 'invalid'
while thingToUpdate not in ['plugins', 'themes']:
    thingToUpdate = input("What do you want to update? (plugins or themes)?")

In [ ]:
with open(f'all_{thingToUpdate}_info.yml', 'r') as file:
    originalEntryInfo = yaml.load(file, Loader=yaml.FullLoader)

In [ ]:
def describe_entry(entry):
    properties = []
    for key, value in entry.items():
        if key in ['description_url', 'download_url']:
            properties.append(f'{key.capitalize()}: <a href="{value}">{value}</a>')
        else:
            properties.append(f'{key.capitalize()}: {value}')
    description = '<br/>'.join(properties)
    return description

In [ ]:
for name, entry in originalEntryInfo.items():
    if 'available' not in entry:
        display(HTML(describe_entry(entry)))
        available = 'invalid'
        while available not in ['Y', 'N']:
            available = input("Make this plugin available? (Y/N)")
        note = input("Add any note about the plugin (justification for availablility)")
        if available == 'Y':
            originalEntryInfo[name]['available'] = True
        else:
            originalEntryInfo[name]['available'] = False
        originalEntryInfo[name]['note'] = note
        with open(f'all_{thingToUpdate}_info.yml', 'w') as file:
            documents = yaml.dump(originalEntryInfo, file)

In [ ]:
with open(f'{thingToUpdate}.yml', 'r') as file:
    cache_config = yaml.load(file, Loader=yaml.FullLoader)

In [ ]:
for name, entry in originalEntryInfo.items():
    if entry['available']:
        if name in cache_config:
            # Plugin exists in cache config
            if cache_config[name]['url'] != entry['download_url']:
                # Download urls differ, need input on which to use
                print(f"Addon download links differ.\nOld: {cache_config[name]['url']}\nNew: {entry['download_url']}")
                do_update = 'invalid'
                while do_update not in ['Y', 'N']:
                    do_update = input("Replace the old link with the new one? (Y/N)")
                if do_update == 'Y':
                    cache_config[name]['url'] = entry['download_url']
                continue
            else:
                # Download urls are the same, nothing to do
                continue
        else:
            # Plugin does not exist in cache config, but is listed as available. Need input on name parameter.
            print(f"Please provide a name for {entry['name']}")
            plugin_folder_name = input("Name for addon folder (if for plugin, needs to match main plugin php file)")
            cache_config[name] = {
                'name': plugin_folder_name,
                'url': entry['download_url']
            }
            continue
    else:
        # Plugin is not listed as available, nothing to do
        continue

In [ ]:
with open(f'{thingToUpdate}.yml', 'w') as file:
    documents = yaml.dump(cache_config, file)